# **Part1&2**

## **匯入模組**

匯入標準庫及第三方模組，用來處理時間、資料結構、路徑管理、HTTP 請求和 HTML 解析等功能。

In [1]:
from datetime import date  # 導入日期模組
import time  # 導入時間模組
from collections import namedtuple  # 導入命名元組
from pathlib import Path  # 導入路徑處理模組
from typing import List  # 導入類型提示
from urllib.parse import urljoin  # 導入URL合併功能
import random  # 導入隨機數模組

import requests  # 導入requests庫，用於發送HTTP請求
from bs4 import BeautifulSoup  # 導入BeautifulSoup庫，用於解析HTML
from requests.adapters import HTTPAdapter  # 導入HTTP適配器
from urllib3.util.retry import Retry  # 導入重試功能
import pandas as pd  # 導入pandas庫，用於數據處理

# SEC搜索API的端點
SEARCHAPIENDPOINT = "https://efts.sec.gov/LATEST/search-index"
# SEC存檔的基本URL
ARCHIVESBASEURL = "https://www.sec.gov/Archives/edgar/data"
# 每次請求之間的休眠時間
SLEEPTIME = 0.2
# 最大重試次數
MAXRETRIES = 10
# 日期格式
DATE_FORMAT_TOKENS = "%Y-%m-%d"
# 查詢的開始日期
AFTER_DATE = date(2000, 1, 1)
# 查詢的結束日期（今天）
BEFORE_DATE = date.today()


## **重試機制**

使用 Retry 來設定 HTTP 請求的重試機制，包括重試的次數、延遲時間及應對的錯誤代碼列表（如403、500等）。

In [2]:
# 設定重試策略
retries = Retry(
    total=MAXRETRIES,  # 設定最大重試次數
    backoff_factor=SLEEPTIME,  # 設定每次重試之間的間隔時間，根據後退因子來計算
    status_forcelist=[403, 500, 502, 503, 504],  # 當響應狀態碼為這些值時，會觸發重試
)


## **定義常數與命名元組**

定義一些檔案名常數，並使用 namedtuple 定義一個結構 FilingMetadata，用來儲存每筆年報的相關資訊。

In [3]:
# 設定根保存文件夾名稱
ROOT_SAVE_FOLDER_NAME = "test"

# 設定提交的完整文檔檔名
FILING_FULL_SUBMISSION_FILENAME = "full-submission.txt"

# 設定提交詳細信息檔名的前綴
FILING_DETAILS_FILENAME_STEM = "filing-details"

# 定義一個命名元組，用來儲存提交的元數據
FilingMetadata = namedtuple(
    "FilingMetadata",  # 命名元組的名稱
    [
        "cik",  # 公司在美國證券交易委員會的中央索引鍵
        "file_date",  # 提交的日期
        "period_end",  # 財務報表期間結束日期
        "accession_number",  # 文件的登記號碼
        "full_submission_url",  # 完整提交文檔的URL
        "filing_details_url",  # 提交詳細信息的URL
        "filing_details_filename",  # 提交詳細信息的文件名
    ],
)



## **建立搜尋請求**

建立搜尋請求的函式，根據公司代號 (ticker)、檔案類型（例如 "10-K"）、開始與結束日期等參數生成搜尋請求。

In [4]:
def form_request(
    ticker: str,  # 股票代碼，例如 "AAPL" 表示蘋果公司
    filing_types: List[str],  # 一個列表，包含要查詢的提交類型，例如 ["10-K", "10-Q"]
    start_date: str,  # 查詢的開始日期，格式為 "YYYY-MM-DD"
    end_date: str,  # 查詢的結束日期，格式為 "YYYY-MM-DD"
    start_index: int,  # 查詢的起始索引，用於分頁
    query: str,  # 用於查詢的關鍵字，通常是附加的搜尋條件
) -> dict:  # 函數返回一個字典
    # 構建請求字典，包含查詢的參數
    request = {
        "dateRange": "custom",  # 指定日期範圍為自定義
        "startdt": start_date,  # 開始日期
        "enddt": end_date,  # 結束日期
        "entityName": ticker,  # 公司名稱或股票代碼
        "forms": filing_types,  # 提交類型
        "from": start_index,  # 起始索引
        "q": query,  # 查詢的關鍵字
    }
    return request  # 返回構建好的請求字典


## **提取年報的元數據**

filing_metadata 函式根據搜尋結果 hit 生成年報的詳細信息，並且構建相關 URL 用來下載完整年報和詳細資料。

In [5]:
def filing_metadata(hit: dict) -> FilingMetadata:
    # 從 hit 字典中提取必要的資料
    accession_number, filing_details_filename = hit["_id"].split(":", 1)  # 分割出 accession number 和 filing details filename
    cik = hit["_source"]["ciks"][-1]  # 提取 CIK (中央索引鍵)
    file_date = hit["_source"]["file_date"]  # 提取文件日期
    period_ending = hit["_source"]["period_ending"]  # 提取期間結束日期
    accession_number_no_dashes = accession_number.replace("-", "", 2)  # 移除 accession number 的前兩個短橫

    # 基本的提交 URL 組合
    submission_base_url = f"{ARCHIVESBASEURL}/{cik}/{accession_number_no_dashes}"
    full_submission_url = f"{submission_base_url}/{accession_number}.txt"  # 完整的提交文件 URL
    filing_details_url = f"{submission_base_url}/{filing_details_filename}"  # 提交詳細信息的 URL

    # 設定提交詳細信息的文件名，將 .htm 擴展名替換為 .html
    filing_details_filename_extension = Path(filing_details_filename).suffix.replace(
        "htm", "html"
    )
    filing_details_filename = (
        f"{FILING_DETAILS_FILENAME_STEM}{filing_details_filename_extension}"  # 使用定義的文件名幹
    )

    # 返回一個 FilingMetadata 實例
    return FilingMetadata(
        cik,
        file_date,
        period_ending,
        accession_number=accession_number,
        full_submission_url=full_submission_url,
        filing_details_url=filing_details_url,
        filing_details_filename=filing_details_filename,
    )

# 用於儲存要下載的 filings 的列表
filings_to_download: List[FilingMetadata] = []


## **抓取年報網址**

- 該函式負責從 SEC API 抓取指定公司、年份和年報類型的結果，並儲存在 filings_to_download 列表中。
- 它會自動處理網路錯誤並重試請求，並支持過濾是否包含修訂檔（/A）。

In [6]:
def get_filing_urls(
    filing_type: str,
    ticker: str,
    num_filings_to_download: int,
    after_date: str,
    before_date: str,
    include_amends: bool,
    query: str = "",
) -> List[FilingMetadata]:
    # 設定起始索引
    start_index = 0

    # 建立一個會話客戶端以管理 HTTP 請求
    client = requests.Session()
    client.mount("http://", HTTPAdapter(max_retries=retries))  # 設定重試機制

    try:
        # 當下載的 filings 數量少於所需數量時，持續請求
        while len(filings_to_download) < num_filings_to_download:
            # 建立請求的有效負載
            payload = form_request(
                ticker,
                [filing_type],
                after_date,
                before_date,
                start_index,
                query
            )
            # 設定請求標頭
            headers = {
                "User-Agent": "XXXXX@gmail.com",
                "Accept-Encoding": "gzip, deflate",
                "Host": "efts.sec.gov",
            }
            # 發送 GET 請求
            resp = client.get(
                SEARCHAPIENDPOINT, params=payload, headers=headers, verify=False
            )
            resp.raise_for_status()  # 檢查請求是否成功
            queryresults = resp.json()  # 解析 JSON 響應
            print(queryresults)  # 輸出查詢結果

            queryhits = queryresults["hits"]["hits"]  # 提取查詢命中結果
            print(queryhits)  # 輸出查詢命中結果

            if not queryhits:  # 如果沒有命中結果，則停止查詢
                break

            # 遍歷所有命中結果
            for hit in queryhits:
                filing_type = hit["_source"]["file_type"]  # 提取提交類型

                is_amend = filing_type[-2:] == "/A"  # 檢查是否為修訂
                if not include_amends and is_amend:  # 如果不包含修訂且為修訂，則跳過
                    continue

                if not is_amend and filing_type != filing_type:  # 如果不是修訂且類型不匹配，則跳過
                    continue

                metadata = filing_metadata(hit)  # 提取元數據
                filings_to_download.append(metadata)  # 將元數據添加到列表中

            # 如果已經下載到所需的 filings 數量，則返回結果
            if len(filings_to_download) == num_filings_to_download:
                return filings_to_download

            query_size = queryresults["query"]["size"]  # 獲取查詢結果的大小
            start_index += query_size  # 更新起始索引，以便進行下一次查詢

            time.sleep(SLEEPTIME)  # 暫停一段時間，以遵循 API 請求限制
    finally:
        client.close()  # 關閉客戶端連接

    return filings_to_download  # 返回已下載的 filings 列表


## **解析 HTML 檔案中的相對路徑**

該函式將 HTML 檔案中的相對 URL 轉換為絕對 URL，確保下載的檔案包含正確的鏈接和圖片資源。
- filing_text：這個變數應該包含了一段從 SEC 申報文件中提取的 HTML 文本。通常這是通過 HTTP 請求從 SEC 網站下載的內容。
- "html.parser"：這個參數指定使用 Python 內建的 HTML 解析器。BeautifulSoup 支持多種解析器，包括 lxml 和 html5lib，這裡使用的是標準的 HTML 解析器。

In [7]:
def resolve_relative_urls(filing_text: str, download_url: str) -> str:
    # 使用 BeautifulSoup 解析 HTML 文本
    soup = BeautifulSoup(filing_text, "html.parser")
    # 獲取基本 URL，去掉下載 URL 的最後一部分以建立基礎路徑
    base_url = f"{download_url.rsplit('/', 1)[0]}/"

    # 遍歷所有帶有 href 屬性的超連結
    for url in soup.find_all("a", href=True):
        # 如果 href 以 # 或 http 開頭，則跳過
        if url["href"].startswith("#") or url["href"].startswith("http"):
            continue
        # 將相對 URL 轉換為絕對 URL
        url["href"] = urljoin(base_url, url["href"])

    # 遍歷所有帶有 src 屬性的圖片
    for image in soup.find_all("img", src=True):
        # 將相對的圖片來源轉換為絕對 URL
        image["src"] = urljoin(base_url, image["src"])

    # 如果 soup 的原始編碼為 None，則返回 soup
    if soup.original_encoding is None:
        return soup

    # 否則，返回以原始編碼編碼的 HTML 字符串
    return soup.encode(soup.original_encoding)


## **下載年報**

In [8]:
def download_filings(
    download_folder: Path,
    ticker: str,
    filing_type: str,
    num_filings_to_download: int,
    after_date: str, 
    before_date: str,
    include_filing_details: bool,
) -> None:
    # 創建一個 requests 的 Session，以便重複使用連接
    client = requests.Session()
    # 設定 HTTP 和 HTTPS 的重試機制
    client.mount("http://", HTTPAdapter(max_retries=retries))
    client.mount("https://", HTTPAdapter(max_retries=retries))
    
    try:
        # 遍歷要下載的每個申報文件
        for filing in filings_to_download:
            try:
                # 嘗試下載完整的申報文件
                download(
                    client,
                    download_folder,
                    ticker,
                    filing.accession_number,
                    filing_type,
                    filing.full_submission_url,
                    FILING_FULL_SUBMISSION_FILENAME,
                )
            except requests.exceptions.HTTPError as e:
                # 如果下載過程中出現 HTTP 錯誤，則跳過該文件
                print(
                    "Skipping full submission download for "
                    f"'{filing.accession_number}' due to network error: {e}."
                )

            # 如果需要下載申報詳細信息
            if include_filing_details:
                try:
                    # 嘗試下載申報詳細信息
                    download(
                        client,
                        download_folder,
                        ticker,
                        filing.accession_number,
                        filing_type,
                        filing.filing_details_url,
                        filing.filing_details_filename,
                        resolve_urls=True,
                    )
                except requests.exceptions.HTTPError as e:
                    # 如果下載過程中出現 HTTP 錯誤，則跳過該文件
                    print(
                        f"Skipping filing detail download for "
                        f"'{filing.accession_number}' due to network error: {e}."
                    )
    finally:
        # 確保在結束時關閉 Session
        client.close()


In [9]:
def download(
    client: requests.Session,
    download_folder: Path,
    ticker: str,
    accession_number: str,
    filing_type: str,
    download_url: str,
    save_filename: str,
    *,
    resolve_urls: bool = False,
) -> None:
    # 設定 HTTP 請求的標頭
    headers = {
        "User-Agent": "XXXXX@gmail.com" ,  # 用戶代理，應用發送請求的身份
        "Accept-Encoding": "gzip, deflate",  # 接受的編碼格式
        "Host": "www.sec.gov",  # 請求的主機
    }
    
    # 發送 GET 請求以獲取文件
    resp = client.get(download_url, headers=headers)
    # 檢查請求是否成功，若不成功則引發異常
    resp.raise_for_status()
    
    # 獲取返回的內容
    filing_text = resp.content
    
    # 如果需要解析相對 URL 並且文件類型是 HTML
    if resolve_urls and Path(save_filename).suffix == ".html":
        filing_text = resolve_relative_urls(filing_text, download_url)

    # 構建保存文件的完整路徑
    save_path = (
        download_folder
        / ROOT_SAVE_FOLDER_NAME
        / ticker
        / filing_type
        / accession_number
        / save_filename
    )
    
    # 創建所有父目錄（如果尚不存在）
    save_path.parent.mkdir(parents=True, exist_ok=True)
    # 寫入獲取的文件內容
    save_path.write_bytes(filing_text)

    # 等待一段時間以避免過於頻繁的請求
    time.sleep(SLEEPTIME)


## **調用函數以獲取指定條件的 10-K 申報文件的 URL**

抓取美國蘋果公司：2012-2023年之間的年報資料

In [14]:
get_filing_urls(
    filing_type="10-K",
    ticker="AAPL",
    num_filings_to_download=10,
    after_date="2012-01-01",
    before_date="2023-01-01",
    include_amends=True,
    query="AAPL",
)

[FilingMetadata(cik='0000320193', file_date='2013-10-30', period_end='2013-09-28', accession_number='0001193125-13-416534', full_submission_url='https://www.sec.gov/Archives/edgar/data/0000320193/000119312513416534/0001193125-13-416534.txt', filing_details_url='https://www.sec.gov/Archives/edgar/data/0000320193/000119312513416534/d590790d10k.htm', filing_details_filename='filing-details.html'),
 FilingMetadata(cik='0000320193', file_date='2017-11-03', period_end='2017-09-30', accession_number='0000320193-17-000070', full_submission_url='https://www.sec.gov/Archives/edgar/data/0000320193/000032019317000070/0000320193-17-000070.txt', filing_details_url='https://www.sec.gov/Archives/edgar/data/0000320193/000032019317000070/a10-k20179302017.htm', filing_details_filename='filing-details.html'),
 FilingMetadata(cik='0000320193', file_date='2021-10-29', period_end='2021-09-25', accession_number='0000320193-21-000105', full_submission_url='https://www.sec.gov/Archives/edgar/data/0000320193/000

## **最後執行**

In [15]:
# 獲取當前工作目錄
cwd = Path.cwd()

# 設定下載目錄
downloader = Path(r"/Users/andrewhsu/Documents/fintech_10_K/intel")

# 調用函數下載指定的 10-K 申報文件
download_filings(
    downloader,
    "AAPL",
    "10-K",
    10,  # 下載的文件數量
    "2012-01-01",  # 開始日期
    "2023-01-01",  # 結束日期
    include_filing_details=True  # 包含詳細文件
)

# **Part3**

In [16]:
import re  # 匹配正則表達式
import glob  # 用於文件匹配
from bs4 import BeautifulSoup  # 用於解析 HTML
import csv  # 用於 CSV 操作
from pathlib import Path  # 用於文件路徑操作

# 獲取所有 HTML 文件的路徑
files = glob.glob(r"/Users/andrewhsu/Documents/fintech_10_K/test/AAPL/10-K/**/*.html", recursive=True)

def parser(text, section):
    """解析文本以提取特定的項目"""
    
    def extract_text(text, item_start, item_end):
        """根據起始和結束標記提取文本"""
        starts = [i.start() for i in item_start.finditer(text)]  # 找到所有起始標記的位置
        ends = [i.start() for i in item_end.finditer(text)]  # 找到所有結束標記的位置
        positions = list()
        
        # 構建起始和結束標記之間的範圍
        for s in starts:
            control = 0
            for e in ends:
                if control == 0:
                    if s < e:
                        control = 1
                        positions.append([s, e])  # 保存範圍
        
        item_length = 0
        item_position = list()
        # 找到最大長度的項目範圍
        for p in positions:
            if (p[1] - p[0]) > item_length:
                item_length = p[1] - p[0]
                item_position = p
        
        # 提取文本
        item_text = text[item_position[0]:item_position[1]]
        return item_text

    # 預設提取的文本
    businessText = riskText = mdaText = "Something went wrong!"

    # 根據 section 提取相應的項目
    if section == 1 or section == 0:
        try:
            item1_start = re.compile("item\s*[1][\.\;\:\-\_]*\s*\\b", re.IGNORECASE)
            item1_end = re.compile("item\s*1a[\.\;\:\-\_]\s*Risk|item\s*2[\.\,\;\:\-\_]\s*Prop", re.IGNORECASE)
            businessText = extract_text(text, item1_start, item1_end)  # 提取 Item 1
        except:
            businessText = "Something went wrong!"  # 捕捉異常

    if section == 2 or section == 0:
        try:
            item1a_start = re.compile("(?<!,\s)item\s*1a[\.\;\:\-\_]\s*Risk", re.IGNORECASE)
            item1a_end = re.compile("item\s*2[\.\;\:\-\_]\s*Prop|item\s*[1]b[\.\;\:\-\_]*\s*\\b", re.IGNORECASE)
            riskText = extract_text(text, item1a_start, item1a_end)  # 提取 Item 1A
        except:
            riskText = "Something went wrong!"

    if section == 3 or section == 0:
        try:
            item7_start = re.compile("item\s*[7][\.\;\:\-\_]*\s*\\bM", re.IGNORECASE)
            item7_end = re.compile("item\s*7a[\.\;\:\-\_]\sQuanti|item\s*8[\.\,\;\:\-\_]\s*", re.IGNORECASE)
            mdaText = extract_text(text, item7_start, item7_end)  # 提取 Item 7
        except:
            mdaText = "Something went wrong!"

    # 根據 section 返回對應的數據
    if section == 0:
        data = [businessText, riskText, mdaText]
    elif section == 1:
        data = [businessText]
    elif section == 2:
        data = [riskText]
    elif section == 3:
        data = [mdaText]
    
    return data

def save_output(parent_dir, index, output):
    """保存提取的項目到文件中"""
    item_names = ["Item1", "Item1a", "Item7"]  # 項目名稱列表
    for item_name, content in zip(item_names, output):
        savepath = Path(parent_dir, f"{index}_{item_name}.txt")  # 設定保存路徑
        with open(savepath, 'w', newline='', encoding="utf-8") as outfile:
            outfile.write(content)  # 寫入文件

# 設定保存的父目錄路徑
parent_dir = r"/Users/andrewhsu/Documents/fintech_10_K/test/AAPL/保存columns"

# 遍歷所有文件並提取內容
for i, file_path in enumerate(files):
    with open(file_path, encoding='utf-8') as f:
        soup = BeautifulSoup(f.read(), features="html.parser")
        for script in soup(["script", "style"]):
            script.extract()  # 移除 script 和 style 標籤
        text = soup.get_text()  # 提取純文本

        # 抓取所有項目
        output = parser(text, 0)

        # 保存提取的項目
        save_output(parent_dir, i, output)
